# Problem 79

### Passcode derivation

A common security method used for online banking is to ask the user for three random characters from a passcode. For example, if the passcode was 531278, they may ask for the 2nd, 3rd, and 5th characters; the expected reply would be: 317.

The text file, keylog.txt, contains fifty successful login attempts.

Given that the three characters are always asked for in order, analyse the file so as to determine the shortest possible secret passcode of unknown length.

### Solution

First we can see that some attempts are repeated, so we can reduce the number of attempts when loading them.

In [1]:
with open('data/p079_keylog.txt') as f:
    attempts = f.read().split()
    attempts = list(set(attempts))
    print len(attempts), 'unique attempts'

33 unique attempts


We are going to convert the 8 digits/characters that are inside the 33 attempts to consecutive indexes of an array.

In [2]:
problem_digits = '01236789'

char_to_idx = {char: idx for idx, char in enumerate(problem_digits)}
idx_to_char = {idx: char for idx, char in enumerate(problem_digits)}

char_to_idx

{'0': 0, '1': 1, '2': 2, '3': 3, '6': 4, '7': 5, '8': 6, '9': 7}

In [3]:
attempts = [[char_to_idx[char] for char in attempt] for attempt in attempts]

We build a dictionary that, for each digit, contains the attempts whose last digit is that digit.

In [4]:
from collections import defaultdict

last_digit_dictionary = defaultdict(list)
for attempt in attempts:
    last_digit_dictionary[attempt[2]].append(attempt)
    
last_digit_dictionary

defaultdict(list,
            {0: [[4, 2, 0],
              [4, 7, 0],
              [5, 7, 0],
              [1, 6, 0],
              [5, 4, 0],
              [6, 7, 0],
              [5, 1, 0],
              [3, 6, 0],
              [1, 4, 0],
              [5, 2, 0],
              [4, 6, 0],
              [2, 7, 0]],
             1: [[5, 3, 1]],
             2: [[5, 4, 2], [3, 4, 2], [1, 4, 2]],
             4: [[3, 1, 4], [5, 3, 4], [5, 1, 4]],
             6: [[3, 1, 6], [3, 4, 6], [5, 1, 6], [1, 4, 6], [5, 2, 6]],
             7: [[3, 1, 7],
              [4, 2, 7],
              [5, 4, 7],
              [3, 6, 7],
              [2, 6, 7],
              [5, 1, 7],
              [1, 2, 7],
              [4, 6, 7],
              [5, 2, 7]]})

Since we have 8 distinct digits, we can build an array that, for each index *i*, tells how many times we can use that digit inside the passcode. The following code tries recursively to add digits to the final passcode and uses the attempts from the input file to prune the search. In fact, when we have inserted the digit *d* for the maximum number of times allowed, we can verify that all the attempts whose last digit is *d* are valid, by checking the digits that have been included before *d*.

In [5]:
def constraint_check(passcode, attempt):
    if len(passcode) < 3:
        return False
    else:
        try:
            i0 = passcode.index(attempt[0])
            i1 = passcode.index(attempt[1])
            return i0 < i1
        except ValueError:
            return False
        

def find_feasible(vector, passcode=[], last_used_digit=-1):
    
    if sum(vector) == 0:
        print 'Solution:', passcode
        return passcode
        
    for digit in [d for d, v in enumerate(vector) if v > 0 and d != last_used_digit]:
        vector2 = vector[::]
        vector2[digit] -= 1
        passcode2 = passcode[::]
        passcode2.append(digit)
        
        still_valid = True
        
        if vector2[digit] == 0:   # constraints
            
            for attempt in last_digit_dictionary[digit]:
                if not constraint_check(passcode2, attempt):
                    still_valid = False
                    print passcode2, 'is not feasible because of attempt', attempt
                    break
                    
        if still_valid:
            print passcode2
            found_solution = find_feasible(vector2, passcode2, digit)
            if found_solution is not None:
                return found_solution
            
    return None

We can find a solution with the shortest possible solution, made of 8 digits:

In [6]:
passcode = find_feasible([1,1,1,1,1,1,1,1])

[0] is not feasible because of attempt [4, 2, 0]
[1] is not feasible because of attempt [5, 3, 1]
[2] is not feasible because of attempt [5, 4, 2]
[3]
[3, 0] is not feasible because of attempt [4, 2, 0]
[3, 1] is not feasible because of attempt [5, 3, 1]
[3, 2] is not feasible because of attempt [5, 4, 2]
[3, 4] is not feasible because of attempt [3, 1, 4]
[3, 5]
[3, 5, 0] is not feasible because of attempt [4, 2, 0]
[3, 5, 1] is not feasible because of attempt [5, 3, 1]
[3, 5, 2] is not feasible because of attempt [5, 4, 2]
[3, 5, 4] is not feasible because of attempt [3, 1, 4]
[3, 5, 6] is not feasible because of attempt [3, 1, 6]
[3, 5, 7] is not feasible because of attempt [3, 1, 7]
[3, 6] is not feasible because of attempt [3, 1, 6]
[3, 7] is not feasible because of attempt [3, 1, 7]
[4] is not feasible because of attempt [3, 1, 4]
[5]
[5, 0] is not feasible because of attempt [4, 2, 0]
[5, 1] is not feasible because of attempt [5, 3, 1]
[5, 2] is not feasible because of attempt [

We then convert the solution from indexes of our vectors to the initial numbers:

In [7]:
print ''.join([idx_to_char[idx] for idx in passcode])

73162890
